## Preparação de dados para o projeto

## 1. Informação Meteorológica 

Fonte: INMET
![inmet.png](inmet.png)

http://www.inmet.gov.br/portal/index.php?r=bdmep/bdmep

BDMEP - INMET\
\
Estação           : SAO PAULO MIR de SANTANA  - SP (OMM: 83781)\
Latitude  (graus) : -23.5\
Longitude (graus) : -46.61\
Altitude  (metros): 792.06\
Estação Operante\
Inicio de operação: 01/12/1945\
Periodo solicitado dos dados: 01/01/2000 a 30/09/2019\
Os dados listados abaixo são os que encontram-se digitados no BDMEP\
\
Hora em UTC\
\
Obs.: Os dados aparecem separados por ; (ponto e vírgula) no formato txt.


In [1]:
# Importando bibliotecas

import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests



In [2]:
inmet = pd.read_csv(r'Inmet.txt',delimiter=';')

In [4]:
inmet.head()

,Estacao,Data,Hora,Precipitacao,TempMaxima,TempMinima,Evaporacao Piche,Unnamed: 7
0,83781,01/01/2000,0,NaN,24.7,NaN,3.8,NaN
1,83781,01/01/2000,1200,16.8,NaN,19.7,NaN,NaN
2,83781,02/01/2000,0,NaN,20.8,NaN,0.9,NaN
3,83781,02/01/2000,1200,49.8,NaN,17.6,NaN,NaN
4,83781,03/01/2000,0,NaN,23.7,NaN,0.6,NaN


In [3]:
inmet = inmet.fillna(0)

In [24]:
inmet[(inmet['Hora']==0) & (inmet['Precipitacao']!=0) ] # precipitacao esta apenas em horário 12

,Estacao,Data,Hora,Precipitacao,TempMaxima,TempMinima,Evaporacao Piche,Unnamed: 7


In [26]:
inmet[(inmet['Hora']==0) & (inmet['TempMinima']!=0) ] # temp minima esta apenas em horário 12

,Estacao,Data,Hora,Precipitacao,TempMaxima,TempMinima,Evaporacao Piche,Unnamed: 7


In [27]:
inmet[(inmet['Hora']==1200) & (inmet['TempMaxima']!=0) ] # temp maxima esta apenas em horário 0

,Estacao,Data,Hora,Precipitacao,TempMaxima,TempMinima,Evaporacao Piche,Unnamed: 7


In [80]:
# Arquivo trabalhado = inmet_df.csv
# Fazendo Group by dos dados por data e exportando para arquivo apenas as colunas que interessam, sem index
inmet.groupby(['Data']).sum().reset_index()[['Data','Precipitacao','TempMinima','TempMaxima']].to_csv('inmet_formatado.csv' ,index=False)

## 2. Informações sobre áreas alagadas

<img style="float: left;" src="cge.jpg">

    Fonte: CGE/SP - Centro de Gerenciamento de Emergências \
    https://www.cgesp.org/v3/
\
\
    Como uma das features da durabilidade do asfalto é a drenagem, buscou-se informação de lugarem com problemas na drenagem, ou seja, regiões de alagamento




Será Feito um scrap , pelo fato da informação histórica ser em função da data digitada 

In [155]:
# Teste de montagem de url
dia = '01' 
mes = '01'
ano = '2019'
url = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=' + dia + '%2F' + mes + '%2F' + ano +'&enviaBusca=Buscar'
url

'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=01%2F01%2F2019&enviaBusca=Buscar'

In [156]:
doc = requests.get(url).content # informações da url

In [157]:
conteudo = BeautifulSoup(doc, 'html.parser') # transformando em html

In [158]:
x = conteudo.find_all('div', class_ = 'content')[0] # separando apenas a parte com dados úteis, indice =0 

In [159]:
#
#x.text.strip() #    'Não há registros de alagamentos para essa data.'

'Zona Norte\n\n\n\r\n\t\t\t\t\tPerus \r\n\t\t\t\t\t\n\n4 pts.\n\n\n\n\n\n\n\nDe 17:16 a 22:29R. CRISPIM DO AMARAL\n\nSentido: CENTRO/BAIRROReferência: ALTURA DO NÚMERO 102\n\n\n\n\n\n\n\n\n\n\nDe 17:33 a 22:29PÇ INACIO DIAS\n\nSentido: AMBOSReferência: RUA CRISPIM DO AMARAL\n\n\n\n\n\n\n\n\n\n\nDe 18:01 a 22:29RUA FRANCIS DE CASTELNAU\n\nSentido: AMBOSReferência: ALTURA DO NUMERO 10\n\n\n\n\n\n\n\n\n\n\nDe 19:04 a 21:09AV. RAIMUNDO PEREIRA DE MAGALHAES\n\nSentido: CENTRO/BAIRROReferência: ALTURA DO NÚMERO 16271\n\n\n\n\n\nCentro\n\n\n\r\n\t\t\t\t\tSé \r\n\t\t\t\t\t\n\n1 pts.\n\n\n\n\n\n\n\nDe 18:34 a 19:23PC. DA BANDEIRA\n\nSentido: CENTRO/BAIRROReferência: AV. VINTE E TRES DE MAIO\n\n\n\n\n\nZona Oeste\n\n\n\r\n\t\t\t\t\tPinheiros \r\n\t\t\t\t\t\n\n1 pts.\n\n\n\n\n\n\n\nDe 19:52 a 21:00AV. DAS NACOES UNIDAS\n\nSentido: INTERLAGOS/CASTELOReferência: AV QUEIROZ FILHO\n\n\n\n\n\nZona Leste\n\n\n\r\n\t\t\t\t\tItaquera \r\n\t\t\t\t\t\n\n3 pts.\n\n\n\n\n\n\n\nDe 16:25 a 17:23AV. JACU PESSEG

In [75]:
z = x.find_all('h1', class_ = 'tit-bairros')
# PENDENTE!!!!!!!!!!!!!!!

[<h1 class="tit-bairros">Zona Norte</h1>,
 <h1 class="tit-bairros">Centro</h1>,
 <h1 class="tit-bairros">Zona Oeste</h1>,
 <h1 class="tit-bairros">Zona Leste</h1>]

In [79]:
y = x.find_all('table', class_ = 'tb-pontos-de-alagamentos')   
len(y) # cada saída é um bairro

6

In [81]:
i=0
y[i]

<table class="tb-pontos-de-alagamentos">
<tr>
<td class="bairro arial-bairros-alag linha-pontilhada">
					Perus 
					<hr/>
</td>
<td class="total-pts arial-bairros-alag">4 pts.</td>
</tr>
<tr>
<td colspan="3">
<div class="ponto-de-alagamento">
<ul>
<li class="inativo-intransitavel" title="Inativo Intransitável"></li>
<li class="espaco"></li>
<li class="arial-descr-alag col-local">De 17:16 a 22:29<br/>R. CRISPIM DO AMARAL</li>
<li class="ln-vert"></li>
<li class="arial-descr-alag">Sentido: CENTRO/BAIRRO<br/>Referência: ALTURA DO NÚMERO 102</li>
</ul>
</div>
</td>
</tr>
<tr>
<td colspan="3">
<div class="ponto-de-alagamento">
<ul>
<li class="inativo-intransitavel" title="Inativo Intransitável"></li>
<li class="espaco"></li>
<li class="arial-descr-alag col-local">De 17:33 a 22:29<br/>PÇ INACIO DIAS</li>
<li class="ln-vert"></li>
<li class="arial-descr-alag">Sentido: AMBOS<br/>Referência: RUA CRISPIM DO AMARAL</li>
</ul>
</div>
</td>
</tr>
<tr>
<td colspan="3">
<div class="ponto-de-alagam

In [84]:
bairro = y[i].find('td', class_ = 'bairro arial-bairros-alag linha-pontilhada').text.strip()
bairro

'Perus'

In [88]:
alagados = y[i].find_all('li', class_ = 'arial-descr-alag')

In [90]:
alagados

[<li class="arial-descr-alag col-local">De 17:16 a 22:29<br/>R. CRISPIM DO AMARAL</li>,
 <li class="arial-descr-alag">Sentido: CENTRO/BAIRRO<br/>Referência: ALTURA DO NÚMERO 102</li>,
 <li class="arial-descr-alag col-local">De 17:33 a 22:29<br/>PÇ INACIO DIAS</li>,
 <li class="arial-descr-alag">Sentido: AMBOS<br/>Referência: RUA CRISPIM DO AMARAL</li>,
 <li class="arial-descr-alag col-local">De 18:01 a 22:29<br/>RUA FRANCIS DE CASTELNAU</li>,
 <li class="arial-descr-alag">Sentido: AMBOS<br/>Referência: ALTURA DO NUMERO 10</li>,
 <li class="arial-descr-alag col-local">De 19:04 a 21:09<br/>AV. RAIMUNDO PEREIRA DE MAGALHAES</li>,
 <li class="arial-descr-alag">Sentido: CENTRO/BAIRRO<br/>Referência: ALTURA DO NÚMERO 16271</li>]

In [89]:
len(alagados)

8

In [130]:
j=0
local = alagados[j].text.replace(alagados[j].next_element,'')

'R. CRISPIM DO AMARAL'

In [131]:
j += 1
referencia = alagados[j].text.replace(alagados[j].next_element,'').replace('Referência: ','')

'ALTURA DO NÚMERO 102'

In [133]:
sentido = alagados[j].next_element.replace('Sentido: ', '')


'CENTRO/BAIRRO'

In [166]:
str(2019)
str(1).zfill(2)

#'Não há registros de alagamentos para essa data.'

'01'

In [174]:
# LOOP onde está sendo executado o scrap e montagem do arquivo com os dados

lista_alagados =[] # sera montada a lista com dicionarios dos pontos alagados

min_ano = 2010 # primeiro ano dos dados 
max_ano = 2019 # último ano dos dados 
ano = min_ano # inicializando a variavel ano 

while ano <= max_ano:      # loop do ano
    mes =1                 #ao iniciar o ano, o mes é 1
    while mes <=12:        # loop do mes
        dia = 1            #ao iniciar o mes, o dia é 1 
        while dia<=31:     # Loop do dia, se o mes não tem 31, retorna informação que nao tem registro de alagamento
            url = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=' + str(dia).zfill(2) + '%2F' + str(mes).zfill(2) + '%2F' + str(ano) +'&enviaBusca=Buscar'
            doc = requests.get(url).content # informações da url
            conteudo = BeautifulSoup(doc, 'html.parser') # transformando em html
            print('fazendo ' + str(dia).zfill(2) + '/' + str(mes).zfill(2) + '/'+ str(ano) )
            try:
                x = conteudo.find_all('div', class_ = 'content')[0] # separando apenas a parte com dados úteis, indice =0 
                if x.text.strip() == 'Não há registros de alagamentos para essa data.': 
                    pass # se não tem informação de alagamento, passa...
                else: # se tem informação de alagamento, busca informação de todos os bairros alagados 
                    y = x.find_all('table', class_ = 'tb-pontos-de-alagamentos' ) # cada item da saída é um bairro
                    for i in range (0, len(y)):
                        bairro = y[i].find('td', class_ = 'bairro arial-bairros-alag linha-pontilhada').text.strip()
                        alagados = y[i].find_all('li', class_ = 'arial-descr-alag')
                        j=0
                        while j < len(alagados): 
                            local = alagados[j].text.replace(alagados[j].next_element,'')
                            j += 1
                            referencia = alagados[j].text.replace(alagados[j].next_element,'').replace('Referência: ','')
                            sentido = alagados[j].next_element.replace('Sentido: ', '')
                            lista_alagados.append({'ano':str(ano), 'mes':str(mes).zfill(2), 'dia':str(dia).zfill(2), 'bairro': bairro, 'local':local , 'referencia': referencia, 'sentido':sentido})
                            j += 1
            except:
                pass
            dia += 1
        mes += 1
    pd.DataFrame(lista_alagados).to_csv('alagados_formatado'+str(ano)+ '.csv' ,index=False)
    ano += 1
  
 #arquivo com a relação de todos os registros de alagamento


fazendo 01/01/2010
fazendo 02/01/2010
fazendo 03/01/2010
fazendo 04/01/2010
fazendo 05/01/2010
fazendo 06/01/2010
fazendo 07/01/2010
fazendo 08/01/2010
fazendo 09/01/2010
fazendo 10/01/2010
fazendo 11/01/2010
fazendo 12/01/2010
fazendo 13/01/2010
fazendo 14/01/2010
fazendo 15/01/2010
fazendo 16/01/2010
fazendo 17/01/2010
fazendo 18/01/2010
fazendo 19/01/2010
fazendo 20/01/2010
fazendo 21/01/2010
fazendo 22/01/2010
fazendo 23/01/2010
fazendo 24/01/2010
fazendo 25/01/2010
fazendo 26/01/2010
fazendo 27/01/2010
fazendo 28/01/2010
fazendo 29/01/2010
fazendo 30/01/2010
fazendo 31/01/2010
fazendo 01/02/2010
fazendo 02/02/2010
fazendo 03/02/2010
fazendo 04/02/2010
fazendo 05/02/2010
fazendo 06/02/2010
fazendo 07/02/2010
fazendo 08/02/2010
fazendo 09/02/2010
fazendo 10/02/2010
fazendo 11/02/2010
fazendo 12/02/2010
fazendo 13/02/2010
fazendo 14/02/2010
fazendo 15/02/2010
fazendo 16/02/2010
fazendo 17/02/2010
fazendo 18/02/2010
fazendo 19/02/2010
fazendo 20/02/2010
fazendo 21/02/2010
fazendo 22/0

fazendo 31/02/2011
fazendo 01/03/2011
fazendo 02/03/2011
fazendo 03/03/2011
fazendo 04/03/2011
fazendo 05/03/2011
fazendo 06/03/2011
fazendo 07/03/2011
fazendo 08/03/2011
fazendo 09/03/2011
fazendo 10/03/2011
fazendo 11/03/2011
fazendo 12/03/2011
fazendo 13/03/2011
fazendo 14/03/2011
fazendo 15/03/2011
fazendo 16/03/2011
fazendo 17/03/2011
fazendo 18/03/2011
fazendo 19/03/2011
fazendo 20/03/2011
fazendo 21/03/2011
fazendo 22/03/2011
fazendo 23/03/2011
fazendo 24/03/2011
fazendo 25/03/2011
fazendo 26/03/2011
fazendo 27/03/2011
fazendo 28/03/2011
fazendo 29/03/2011
fazendo 30/03/2011
fazendo 31/03/2011
fazendo 01/04/2011
fazendo 02/04/2011
fazendo 03/04/2011
fazendo 04/04/2011
fazendo 05/04/2011
fazendo 06/04/2011
fazendo 07/04/2011
fazendo 08/04/2011
fazendo 09/04/2011
fazendo 10/04/2011
fazendo 11/04/2011
fazendo 12/04/2011
fazendo 13/04/2011
fazendo 14/04/2011
fazendo 15/04/2011
fazendo 16/04/2011
fazendo 17/04/2011
fazendo 18/04/2011
fazendo 19/04/2011
fazendo 20/04/2011
fazendo 21/0

fazendo 29/04/2012
fazendo 30/04/2012
fazendo 31/04/2012
fazendo 01/05/2012
fazendo 02/05/2012
fazendo 03/05/2012
fazendo 04/05/2012
fazendo 05/05/2012
fazendo 06/05/2012
fazendo 07/05/2012
fazendo 08/05/2012
fazendo 09/05/2012
fazendo 10/05/2012
fazendo 11/05/2012
fazendo 12/05/2012
fazendo 13/05/2012
fazendo 14/05/2012
fazendo 15/05/2012
fazendo 16/05/2012
fazendo 17/05/2012
fazendo 18/05/2012
fazendo 19/05/2012
fazendo 20/05/2012
fazendo 21/05/2012
fazendo 22/05/2012
fazendo 23/05/2012
fazendo 24/05/2012
fazendo 25/05/2012
fazendo 26/05/2012
fazendo 27/05/2012
fazendo 28/05/2012
fazendo 29/05/2012
fazendo 30/05/2012
fazendo 31/05/2012
fazendo 01/06/2012
fazendo 02/06/2012
fazendo 03/06/2012
fazendo 04/06/2012
fazendo 05/06/2012
fazendo 06/06/2012
fazendo 07/06/2012
fazendo 08/06/2012
fazendo 09/06/2012
fazendo 10/06/2012
fazendo 11/06/2012
fazendo 12/06/2012
fazendo 13/06/2012
fazendo 14/06/2012
fazendo 15/06/2012
fazendo 16/06/2012
fazendo 17/06/2012
fazendo 18/06/2012
fazendo 19/0

fazendo 28/06/2013
fazendo 29/06/2013
fazendo 30/06/2013
fazendo 31/06/2013
fazendo 01/07/2013
fazendo 02/07/2013
fazendo 03/07/2013
fazendo 04/07/2013
fazendo 05/07/2013
fazendo 06/07/2013
fazendo 07/07/2013
fazendo 08/07/2013
fazendo 09/07/2013
fazendo 10/07/2013
fazendo 11/07/2013
fazendo 12/07/2013
fazendo 13/07/2013
fazendo 14/07/2013
fazendo 15/07/2013
fazendo 16/07/2013
fazendo 17/07/2013
fazendo 18/07/2013
fazendo 19/07/2013
fazendo 20/07/2013
fazendo 21/07/2013
fazendo 22/07/2013
fazendo 23/07/2013
fazendo 24/07/2013
fazendo 25/07/2013
fazendo 26/07/2013
fazendo 27/07/2013
fazendo 28/07/2013
fazendo 29/07/2013
fazendo 30/07/2013
fazendo 31/07/2013
fazendo 01/08/2013
fazendo 02/08/2013
fazendo 03/08/2013
fazendo 04/08/2013
fazendo 05/08/2013
fazendo 06/08/2013
fazendo 07/08/2013
fazendo 08/08/2013
fazendo 09/08/2013
fazendo 10/08/2013
fazendo 11/08/2013
fazendo 12/08/2013
fazendo 13/08/2013
fazendo 14/08/2013
fazendo 15/08/2013
fazendo 16/08/2013
fazendo 17/08/2013
fazendo 18/0

fazendo 26/08/2014
fazendo 27/08/2014
fazendo 28/08/2014
fazendo 29/08/2014
fazendo 30/08/2014
fazendo 31/08/2014
fazendo 01/09/2014
fazendo 02/09/2014
fazendo 03/09/2014
fazendo 04/09/2014
fazendo 05/09/2014
fazendo 06/09/2014
fazendo 07/09/2014
fazendo 08/09/2014
fazendo 09/09/2014
fazendo 10/09/2014
fazendo 11/09/2014
fazendo 12/09/2014
fazendo 13/09/2014
fazendo 14/09/2014
fazendo 15/09/2014
fazendo 16/09/2014
fazendo 17/09/2014
fazendo 18/09/2014
fazendo 19/09/2014
fazendo 20/09/2014
fazendo 21/09/2014
fazendo 22/09/2014
fazendo 23/09/2014
fazendo 24/09/2014
fazendo 25/09/2014
fazendo 26/09/2014
fazendo 27/09/2014
fazendo 28/09/2014
fazendo 29/09/2014
fazendo 30/09/2014
fazendo 31/09/2014
fazendo 01/10/2014
fazendo 02/10/2014
fazendo 03/10/2014
fazendo 04/10/2014
fazendo 05/10/2014
fazendo 06/10/2014
fazendo 07/10/2014
fazendo 08/10/2014
fazendo 09/10/2014
fazendo 10/10/2014
fazendo 11/10/2014
fazendo 12/10/2014
fazendo 13/10/2014
fazendo 14/10/2014
fazendo 15/10/2014
fazendo 16/1

fazendo 24/10/2015
fazendo 25/10/2015
fazendo 26/10/2015
fazendo 27/10/2015
fazendo 28/10/2015
fazendo 29/10/2015
fazendo 30/10/2015
fazendo 31/10/2015
fazendo 01/11/2015
fazendo 02/11/2015
fazendo 03/11/2015
fazendo 04/11/2015
fazendo 05/11/2015
fazendo 06/11/2015
fazendo 07/11/2015
fazendo 08/11/2015
fazendo 09/11/2015
fazendo 10/11/2015
fazendo 11/11/2015
fazendo 12/11/2015
fazendo 13/11/2015
fazendo 14/11/2015
fazendo 15/11/2015
fazendo 16/11/2015
fazendo 17/11/2015
fazendo 18/11/2015
fazendo 19/11/2015
fazendo 20/11/2015
fazendo 21/11/2015
fazendo 22/11/2015
fazendo 23/11/2015
fazendo 24/11/2015
fazendo 25/11/2015
fazendo 26/11/2015
fazendo 27/11/2015
fazendo 28/11/2015
fazendo 29/11/2015
fazendo 30/11/2015
fazendo 31/11/2015
fazendo 01/12/2015
fazendo 02/12/2015
fazendo 03/12/2015
fazendo 04/12/2015
fazendo 05/12/2015
fazendo 06/12/2015
fazendo 07/12/2015
fazendo 08/12/2015
fazendo 09/12/2015
fazendo 10/12/2015
fazendo 11/12/2015
fazendo 12/12/2015
fazendo 13/12/2015
fazendo 14/1

fazendo 22/12/2016
fazendo 23/12/2016
fazendo 24/12/2016
fazendo 25/12/2016
fazendo 26/12/2016
fazendo 27/12/2016
fazendo 28/12/2016
fazendo 29/12/2016
fazendo 30/12/2016
fazendo 31/12/2016
fazendo 01/01/2017
fazendo 02/01/2017
fazendo 03/01/2017
fazendo 04/01/2017
fazendo 05/01/2017
fazendo 06/01/2017
fazendo 07/01/2017
fazendo 08/01/2017
fazendo 09/01/2017
fazendo 10/01/2017
fazendo 11/01/2017
fazendo 12/01/2017
fazendo 13/01/2017
fazendo 14/01/2017
fazendo 15/01/2017
fazendo 16/01/2017
fazendo 17/01/2017
fazendo 18/01/2017
fazendo 19/01/2017
fazendo 20/01/2017
fazendo 21/01/2017
fazendo 22/01/2017
fazendo 23/01/2017
fazendo 24/01/2017
fazendo 25/01/2017
fazendo 26/01/2017
fazendo 27/01/2017
fazendo 28/01/2017
fazendo 29/01/2017
fazendo 30/01/2017
fazendo 31/01/2017
fazendo 01/02/2017
fazendo 02/02/2017
fazendo 03/02/2017
fazendo 04/02/2017
fazendo 05/02/2017
fazendo 06/02/2017
fazendo 07/02/2017
fazendo 08/02/2017
fazendo 09/02/2017
fazendo 10/02/2017
fazendo 11/02/2017
fazendo 12/0

fazendo 19/02/2018
fazendo 20/02/2018
fazendo 21/02/2018
fazendo 22/02/2018
fazendo 23/02/2018
fazendo 24/02/2018
fazendo 25/02/2018
fazendo 26/02/2018
fazendo 27/02/2018
fazendo 28/02/2018
fazendo 29/02/2018
fazendo 30/02/2018
fazendo 31/02/2018
fazendo 01/03/2018
fazendo 02/03/2018
fazendo 03/03/2018
fazendo 04/03/2018
fazendo 05/03/2018
fazendo 06/03/2018
fazendo 07/03/2018
fazendo 08/03/2018
fazendo 09/03/2018
fazendo 10/03/2018
fazendo 11/03/2018
fazendo 12/03/2018
fazendo 13/03/2018
fazendo 14/03/2018
fazendo 15/03/2018
fazendo 16/03/2018
fazendo 17/03/2018
fazendo 18/03/2018
fazendo 19/03/2018
fazendo 20/03/2018
fazendo 21/03/2018
fazendo 22/03/2018
fazendo 23/03/2018
fazendo 24/03/2018
fazendo 25/03/2018
fazendo 26/03/2018
fazendo 27/03/2018
fazendo 28/03/2018
fazendo 29/03/2018
fazendo 30/03/2018
fazendo 31/03/2018
fazendo 01/04/2018
fazendo 02/04/2018
fazendo 03/04/2018
fazendo 04/04/2018
fazendo 05/04/2018
fazendo 06/04/2018
fazendo 07/04/2018
fazendo 08/04/2018
fazendo 09/0

fazendo 18/04/2019
fazendo 19/04/2019
fazendo 20/04/2019
fazendo 21/04/2019
fazendo 22/04/2019
fazendo 23/04/2019
fazendo 24/04/2019
fazendo 25/04/2019
fazendo 26/04/2019
fazendo 27/04/2019
fazendo 28/04/2019
fazendo 29/04/2019
fazendo 30/04/2019
fazendo 31/04/2019
fazendo 01/05/2019
fazendo 02/05/2019
fazendo 03/05/2019
fazendo 04/05/2019
fazendo 05/05/2019
fazendo 06/05/2019
fazendo 07/05/2019
fazendo 08/05/2019
fazendo 09/05/2019
fazendo 10/05/2019
fazendo 11/05/2019
fazendo 12/05/2019
fazendo 13/05/2019
fazendo 14/05/2019
fazendo 15/05/2019
fazendo 16/05/2019
fazendo 17/05/2019
fazendo 18/05/2019
fazendo 19/05/2019
fazendo 20/05/2019
fazendo 21/05/2019
fazendo 22/05/2019
fazendo 23/05/2019
fazendo 24/05/2019
fazendo 25/05/2019
fazendo 26/05/2019
fazendo 27/05/2019
fazendo 28/05/2019
fazendo 29/05/2019
fazendo 30/05/2019
fazendo 31/05/2019
fazendo 01/06/2019
fazendo 02/06/2019
fazendo 03/06/2019
fazendo 04/06/2019
fazendo 05/06/2019
fazendo 06/06/2019
fazendo 07/06/2019
fazendo 08/0

In [173]:
pd.DataFrame(lista_alagados).to_csv('alagados_formatado' + str(ano)+ '.csv' ,index=False)

In [168]:
lista_alagados

[]